In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [2]:
const WN_PATH="data/corpora/WordNet-2.1/"
db = DB(WN_PATH)

WordNet.DB

In [3]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [4]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

In [5]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


punctuation_space_tokenize (generic function with 1 method)

In [28]:
function load_sense_key_map(wn_basedir)
    path=joinpath(wn_basedir, "dict","index.sense")
    sense_key_map = Dict{Tuple{Int64,String},String}()
    for line in eachline(path)
        full_sense_key, offset_str, sense_num_str, tagcount_str = split(line) 
        lemma_name = first(split(full_sense_key,'%'))
        sense_offset = parse(Int64, offset_str)
        index = (sense_offset,lemma_name)
        @assert(!haskey(sense_key_map, index))
        sense_key_map[index] = full_sense_key
    end
    sense_key_map
end


function sense_key(lem::Lemma, ss::Synset,sense_key_map)
    sense_key_map[(ss.offset,lem.word)]
end
function sense_keys(lem::Lemma,sense_key_map)
    [sense_key_map[(ss_offset,lem.word)] for ss_offset in lem.synset_offsets ]
end

sense_keys (generic function with 1 method)

In [7]:
sense_key_map = load_sense_key_map(WN_PATH);

In [8]:
sense_key(l1,s1, sense_key_map)
sense_keys(db['a',"six"],sense_key_map)

LoadError: LoadError: UndefVarError: l1 not defined
while loading In[8], in expression starting on line 1

In [9]:
sense_key_map[2134693,"discover"]

In [ ]:
using LightXML

In [ ]:
immutable WsdChallenge
    id::String
    word::String
    lemma::String
    pos::Char
    context::Vector{String}
end

function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = first(attribute(lemma_node,"pos"))
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(WsdChallenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges() |> collect;

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:0 [inlined]
 in content(::LightXML.XMLNode) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:172
 in (::##3#5{LightXML.XMLElement})() at ./In[11]:18
while loading In[11], in expression starting on line 32
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:0 [inlined]
 in #attribute#1(::Bool, ::Function, ::LightXML.XMLElement, ::String) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:227
 in (::##3#5{LightXML.XMLElement})() at ./In[11]:22
while loading In[11], in expression starting on line 32


In [ ]:
immutable WsdSolution
    id
    lemma
    pos
    solutions
end

function load_solutions(key_file="data/corpora/wsd/semeval2007_t7/key/dataset21.test.key")
    map(eachline(key_file)) do line
        line_data, comment = split(line,"!!")
        fields = split(line_data)
        doc_id = fields[1]
        instance_id = fields[2]
        solutions = fields[3:end]
        
        lemma,pos = match(r"lemma=(.*)#(.)", comment).captures
        WsdSolution(instance_id, lemma, pos, solutions)
    end
end

solutions = load_solutions();

In [ ]:
using Query

In [ ]:
challenge = challenges[1]

WsdChallenge("d001.s001.t001","editorial","editorial",'n',String["your","oct","6","the","ill","homeless","referred","to","research","by","us","and","six","of","our","colleagues","that","was","reported","in","the","sept","8","issue","of","the","journal","of","the","american","medical","association"])

In [25]:
function lexically_informed_embeddings(db,ee, word, pos)
    source_wvs = ee.embedding[word]
    lemma = db[pos, word]
    target_synsets::Vector{Synset} = synsets(db, db[pos, word])
    if length(source_wvs) == 1
        #Use MCWS
        score, index = findmax((get(ss.word_counts,word,0)+0.1sum(values(ss.word_counts)))
                                for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
    end
            
    embeddings = Vector{eltype(source_wvs)}(length(target_synsets))
    for (ii,synset) in enumerate(target_synsets)
        context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
        embeddings[ii] = synthesize_embedding(ee,context, word)
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings (generic function with 1 method)

In [29]:
function supervised_wsd(challenge::WsdChallenge, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee, challenge.word, challenge.pos)
    sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
    synset = target_synsets[sense_index]
    sense_key(lemma, synset, sense_key_map)
end

supervised_wsd (generic function with 1 method)

In [54]:
ii=17
supervised_wsd(challenges[ii], ee,db,sense_key_map) |> println
solutions[ii] |> println

found%2:41:00::
WsdSolution("d001.s002.t008","find","v",SubString{String}["find%2:31:10::","find%2:30:13::","find%2:39:05::","find%2:39:02::","find%2:31:09::","find%2:32:01::","find%2:32:00::","find%2:39:01::"])


In [31]:
solutions


2269-element Array{WsdSolution,1}:
 WsdSolution("d001.s001.t001","editorial","n",SubString{String}["editorial%1:10:00::"])                                                                                                                                                                                                                                                                                            
 WsdSolution("d001.s001.t002","Ill","a",SubString{String}["ill%3:00:01::"])                                                                                                                                                                                                                                                                                                        
 WsdSolution("d001.s001.t003","Homeless","n",SubString{String}["homeless%1:14:00::","homeless%1:18:00::"])                                                                                                                   

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [ ]:
type LazyLexicallyInformedWordSenseEmbedding <: WordSenseEmbedding
    embedding::LazyDict{String, Vector{Vector{Float32}}} #[Word][sense_id]=sense embedding vector
    classification_tree::TreeNode
    codebook::Dict{String, Vector{Int64}}
end

In [ ]:
type LazyDict{K,V} <: Associative{K,V}
    key_to_val::Function
    backing::Dict{K,V}
end
Base.getindex{K,V}(h::LazyDict{K,V}, key) = get!(()->h.key_to_val(key), h.backing, key, )


function  LazyLexicallyInformedWordSenseEmbedding(source::WordSenseEmbedding, wn::DB)
    LazyLexicallyInformedWordSenseEmbedding(
        LazyDict()
        source.classification_tree,
        source.distribution,
        source.codebook
    )
    
end